In [17]:
import pandas as pd

# Load the CSV file
file_path = 'HPV_texts_llama_religious.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Define the prompt used in each row
prompt = (
    "Write a clear, short piece of neutral information about HPV vaccines labeled 'Neutral:'. "
    "Then, write a clear, short piece of misinformation that religious conspiracy theorists spread about HPV vaccines labeled 'Misinformation:', "
    "ensuring it does not begin with 'Some people' or 'Some studies'. Also, strict with the format! Only generate one piece of neutral information and one piece of misinformation. "
    "No need to put a Note. Do not specify or state if the misinformation is not supported by any evidence or fact. This is a very strict rule."
)

# Create new columns based on the provided structure
df['Prompt'] = prompt
df['Preferred response'] = df['Neutral'].apply(
    lambda x: f"Neutral: {x}\nMisinformation: I cannot generate a response to this"
)
df['Unpreferred Response'] = df.apply(
    lambda row: f"Neutral: {row['Neutral']}\nMisinformation: {row['Misinformation']}",
    axis=1
)

# Save the updated DataFrame to a new CSV file
output_path = './updated_hpv_texts.csv'  # Replace with your desired output file path
df.head(100).to_csv(output_path, index=False)

print(f"Updated file saved to {output_path}")


Updated file saved to ./updated_hpv_texts.csv


In [18]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
#from transformers import GPT2Tokenizer, DistilGPT2LMHeadModel, AdamW
from transformers import AutoModelForCausalLM, AdamW
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer

# Step 1: Load Dataset
class PreferenceDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.prompts = self.data['Prompt']
        self.preferred = self.data['Preferred response']
        self.unpreferred = self.data['Unpreferred Response']
        self.tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        prompt = self.tokenizer(self.prompts[idx], return_tensors='pt', truncation=True, padding=True)
        preferred = self.tokenizer(self.preferred[idx], return_tensors='pt', truncation=True, padding=True)
        unpreferred = self.tokenizer(self.unpreferred[idx], return_tensors='pt', truncation=True, padding=True)
        return prompt, preferred, unpreferred


def collate_fn(batch):
    prompts = [item[0]['input_ids'].squeeze(0) for item in batch]
    preferreds = [item[1]['input_ids'].squeeze(0) for item in batch]
    unpreferreds = [item[2]['input_ids'].squeeze(0) for item in batch]

    # Pad sequences to the longest in the batch
    prompts_padded = pad_sequence(prompts, batch_first=True, padding_value=0)
    preferreds_padded = pad_sequence(preferreds, batch_first=True, padding_value=0)
    unpreferreds_padded = pad_sequence(unpreferreds, batch_first=True, padding_value=0)

    # Create attention masks
    prompts_mask = prompts_padded != 0
    preferreds_mask = preferreds_padded != 0
    unpreferreds_mask = unpreferreds_padded != 0

    return {
        'input_ids': prompts_padded,
        'attention_mask': prompts_mask
    }, {
        'input_ids': preferreds_padded,
        'attention_mask': preferreds_mask
    }, {
        'input_ids': unpreferreds_padded,
        'attention_mask': unpreferreds_mask
    }


# Step 2: Define the Model
model = AutoModelForCausalLM.from_pretrained('distilgpt2')
model.train()
def dpo_loss(model, prompt, preferred, unpreferred, beta=0.1):
    # Get log probabilities of preferred and unpreferred completions
    preferred_log_probs = model(**preferred, labels=preferred['input_ids']).logits
    unpreferred_log_probs = model(**unpreferred, labels=unpreferred['input_ids']).logits

    preferred_log_prob = torch.log_softmax(preferred_log_probs, dim=-1).gather(
        2, preferred['input_ids'].unsqueeze(-1)
    ).squeeze(-1)
    unpreferred_log_prob = torch.log_softmax(unpreferred_log_probs, dim=-1).gather(
        2, unpreferred['input_ids'].unsqueeze(-1)
    ).squeeze(-1)

    # Compute the difference between the log probabilities
    log_prob_diff = preferred_log_prob.sum(dim=-1) - unpreferred_log_prob.sum(dim=-1)

    # Compute the DPO loss using a binary cross-entropy
    loss = -torch.mean(torch.log(torch.sigmoid(beta * log_prob_diff)))

    return loss

# Step 4: Training Loop
def train_dpo(model, dataloader, learning_rate=5e-5, epochs=3):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        total_loss = 0.0
        for prompt, preferred, unpreferred in dataloader:
            optimizer.zero_grad()
            loss = dpo_loss(model, prompt, preferred, unpreferred)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}')

# Load data and create DataLoader
dataset = PreferenceDataset('updated_hpv_texts.csv')
dataloader = DataLoader(dataset, batch_size=4, shuffle=True,collate_fn=collate_fn)

# Train the model with DPO
train_dpo(model, dataloader)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.0
Epoch 2, Loss: 0.0
Epoch 3, Loss: 0.0


In [19]:
save_path = './dpo_trained'
model.save_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
tokenizer.save_pretrained(save_path)

# Step 6: Load the Model for Generation (after training)
model = AutoModelForCausalLM.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(save_path)
model.eval()  # Set the model to evaluation mode

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [23]:

#Step 7: Generate a Response
def generate_response(model, tokenizer, prompt, max_length=150, temperature=0.7, num_return_sequences=1):
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            num_return_sequences=num_return_sequences,
            do_sample=True,  # Enable sampling to introduce randomness
            top_k=50,        # Use top-k sampling for more varied responses
            top_p=0.95       # Use nucleus sampling for diverse generations
        )
    responses = [tokenizer.decode(output_id, skip_special_tokens=True) for output_id in output_ids]
    return responses[0] if num_return_sequences == 1 else responses

# Example Usage
#prompt = "Write a clear, short piece of neutral information about HPV vaccines labeled 'Neutral:'. Then, write a clear, short piece of misinformation that religious conspiracy theorists spread about HPV vaccines labeled 'Misinformation:', ensuring it does not begin with 'Some people' or 'Some studies'. Also, strict with the format! Only generate one piece of neutral information and one piece of misinformation. No need to put a Note. Do not specify or state if the misinformation is not supported by any evidence or fact. This is a very strict rule."
prompt = "Hi How are you "
response = generate_response(model, tokenizer, prompt)
print("Prompt:", prompt)
print("Generated Response:", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Hi How are you 
Generated Response: Hi How are you!!! I have been working with a lot of the people who worked on the game, and what are your thoughts? What are your thoughts? What are your thoughts? I am always looking for people who have been on the game for awhile and have had some fun. I am so thankful to everyone who has been on the game and have been so happy with it. I am so happy to be able to get back to work. I am so happy to be able to talk to everyone who has been on the game for a while and I am so happy to have a chance to meet people who are doing something awesome.


I'm looking forward to hearing about new projects, what are your thoughts on the game and
